In [11]:
import json
import pandas as pd
import statistics
from tqdm import tqdm
import math
import operator
import os

In [83]:
db_3000.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 4 columns):
Unnamed: 0             898 non-null int64
texts_3000             898 non-null object
same_word_list_3000    898 non-null object
percent_3000           898 non-null object
dtypes: int64(1), object(3)
memory usage: 28.1+ KB


In [11]:
db_3000 = pd.read_csv("3000.csv")
db_3000.head()

,Unnamed: 0,texts_3000,same_word_list_3000,percent_3000
0,0,Указом президента России Бориса Ельцина внесен...,"{'отмечать', 'изменение', 'иностранный', 'роль...","71,25"
1,1,В Харькове неизвестные изуродовали лицо депута...,"{'регион', 'серьезный', 'агентство', 'являться...","72,22222222"
2,2,"Руководитель следствия по делу ""Аэрофлота"" ...","{'ставить', 'ход', 'фирма', 'отказываться', 'с...",74
3,3,Постоянный экономический рост позволит Китаю к...,"{'внутренний', 'момент', 'сфера', 'слово', 'мн...","72,60273973"
4,4,В одном из ночных клубов Хмельницкого группа п...,"{'отмечать', 'повод', 'смерть', 'свидетель', '...","70,68965517"


In [13]:
trigramms_db = pd.read_csv("word_db.csv", header = None)
trigramms_db.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,0.025718,-0.004776,0.031789,0.018760,-0.015405,-0.078441,-0.002449,-0.008062,-0.001146,0.010007,...,0.017842,-0.041569,0.077341,0.012496,0.049400,-0.037332,0.001658,0.010653,-0.036526,-0.000000
1,0.018746,0.003426,0.020213,0.056091,-0.006876,-0.030144,0.021508,-0.038640,-0.008778,0.014135,...,0.000866,-0.069830,0.107995,0.041295,0.074002,-0.027547,-0.023377,-0.026141,-0.028869,-0.062423
2,0.030954,0.012660,-0.013010,0.058922,-0.015895,-0.004156,0.043479,-0.097470,0.013334,-0.000223,...,-0.012979,-0.054481,0.129445,0.047809,0.064443,-0.009311,-0.068905,-0.063734,-0.017192,-0.050362
3,0.029982,0.046834,-0.016939,0.034865,-0.004499,-0.025411,0.057243,-0.126531,0.015994,-0.012146,...,-0.063004,0.012827,0.101894,0.074595,0.044913,-0.022268,-0.048629,-0.074751,0.018179,-0.035168
4,0.023059,0.052791,-0.031716,0.051026,-0.040494,-0.007667,0.052596,-0.087467,-0.004326,0.018472,...,-0.052820,0.033877,0.062752,0.100702,0.030436,0.018350,-0.031551,-0.050780,0.061213,-0.059490


In [16]:
trigramms_list = []
with open("trigramm_db.txt", "r", encoding = "utf-8") as f:
    for trig in f.readlines():
        trigramms_list.append(trig[:-1])

In [41]:
trigramms_list[338]

'и Александр Лукашенко'

In [20]:
len(trigramms_db), len(trigramms_list)

(1070, 1070)

In [44]:
def trigramm_recommendation_from_marked_dataset(recommended_text_precalculated_trig_similarity, words_competence_db, debug = False):
    with open(recommended_text_precalculated_trig_similarity, "r", encoding = "utf-8") as f:
        trigr_similarity_dict = json.load(f)
        whole_text_vocab_recommendation_dict = {}
        for trigramm in trigr_similarity_dict.keys():
            if debug: print(trigramm, "==>", trigr_similarity_dict[trigramm])
            understdanding_prob_list = []
            if (trigr_similarity_dict[trigramm]):
                for trgrm in trigr_similarity_dict[trigramm]:
                    assert abs(list(words_competence_db.iloc[trgrm[0][0]])[-1]) == trgrm[2]
                    understdanding_prob = list(words_competence_db.iloc[trgrm[0][0]])[-1]
                    understdanding_prob_list.append(understdanding_prob)
                understdanding_prob_mean = statistics.mean(understdanding_prob_list) 
                whole_text_vocab_recommendation_dict[trigramm] = understdanding_prob
                
        positive_understanding_prob = 0
        whole_understading_prob = 0 
        for trigramm in whole_text_vocab_recommendation_dict.keys():
            if debug:print(trigramm , "===>", whole_text_vocab_recommendation_dict[trigramm],'\n')
            if (whole_text_vocab_recommendation_dict[trigramm] > 0):
                positive_understanding_prob += whole_text_vocab_recommendation_dict[trigramm]
                whole_understading_prob += abs(whole_text_vocab_recommendation_dict[trigramm])
            elif(whole_text_vocab_recommendation_dict[trigramm] < 0):
                whole_understading_prob += abs(whole_text_vocab_recommendation_dict[trigramm])
            if debug:print(positive_understanding_prob, whole_understading_prob)
        understanding_prob = positive_understanding_prob/whole_understading_prob
        return understanding_prob
trigramm_recommendation_from_marked_dataset("./words_features_json/text_0.json", trigramms_db)

0.4800312225948592

In [45]:
sentence_db = pd.read_csv("sentence_db.csv", header = None)
sentence_db.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.000000,0.0,0.213510,0.242857,0.0
1,0.000000,0.000000,0.024805,0.024805,0.0,0.000000,0.277500,0.0
2,0.000000,0.066667,0.000000,0.000000,0.0,0.000000,0.250000,1.0
3,0.076923,0.000000,0.000000,0.000000,0.0,0.138684,0.200000,1.0
4,0.000000,0.071429,0.000000,0.000000,0.0,0.000000,0.238462,1.0


In [92]:
def sentence_recommendation_from_marked_dataset(recommended_text_precalculated_sentences_similarity, sentence_competence_db, debug = False):
    with open(recommended_text_precalculated_sentences_similarity, "r", encoding = "utf-8") as f:
        sentences_similarity_dict = json.load(f)
        whole_text_sent_understanding_prob_list = []
        for rec_sentence_sentence_index in sentences_similarity_dict.keys():
            if debug: print(rec_sentence_sentence_index, "==>", sentences_similarity_dict[rec_sentence_sentence_index])
            if(sentences_similarity_dict[rec_sentence_sentence_index]):
                sum_similarity = 0 
                sentence_understanding_similarity_list = []
                for similar_sentence_properties in sentences_similarity_dict[rec_sentence_sentence_index]:
                    sent_ind= similar_sentence_properties[0]
                    marked_understanding = list(sentence_competence_db.iloc[sent_ind])[-1]
                    sentence_similarity = similar_sentence_properties[1]
                    sum_similarity += sentence_similarity
                    sentence_understanding_similarity_list.append([sentence_similarity, marked_understanding])
                if debug:print(sentence_understanding_similarity_list)
                
                #normalize similarity
                for similarity_ind in range(len(sentence_understanding_similarity_list)):
                    sentence_understanding_similarity_list[similarity_ind][0] /=sum_similarity
                if debug:print(sentence_understanding_similarity_list)
                
                #math probability of sentence understanding
                math_prob = 0
                for similarity_ind in range(len(sentence_understanding_similarity_list)):
                    m_e = float(sentence_understanding_similarity_list[similarity_ind][0])
                    mark = float(sentence_understanding_similarity_list[similarity_ind][1])
                    math_prob +=  m_e* mark
                    if debug: print(sentence_understanding_similarity_list[similarity_ind][0], sentence_understanding_similarity_list[similarity_ind][1], m_e* mark)
                if debug: print("final",math_prob)
                whole_text_sent_understanding_prob_list.append(math_prob)
        if (whole_text_sent_understanding_prob_list):
            final_sent_understanding_prediction = statistics.mean(whole_text_sent_understanding_prob_list)
            return final_sent_understanding_prediction
        else:
            return 0
            
sentence_recommendation_from_marked_dataset("./sentence_features_json/text_0.json", sentence_db)

0.6522199313538282

In [67]:
text_db = pd.read_csv("text_db.csv", header = None)
text_db.head()

,0,1,2,3,4,5,6,7,8,9
0,0.6503,0.65,0.200000,0.200000,0.007800,0.007800,0.000000,0.093184,0.237405,0.50
1,0.5626,0.72,0.153846,0.307692,0.014890,0.011259,0.000000,0.169718,0.201980,0.40
2,0.4951,0.83,0.166667,0.000000,0.028309,0.012447,0.000000,0.170251,0.190485,0.50
3,0.6321,0.66,0.000000,0.250000,0.013009,0.018568,0.000000,0.223918,0.214968,0.25
4,0.6601,0.62,0.142857,0.214286,0.028585,0.004112,0.004621,0.106511,0.206026,0.50


In [81]:
def text_recommendation_from_marked_dataset(recommended_text_precalculated_text_feat_similarity, text_feat_competence_db, debug = False):
    with open(recommended_text_precalculated_text_feat_similarity, "r", encoding = "utf-8") as f:
        text_similarity_dict = json.load(f)
        if text_similarity_dict:
            sum_similarity = 0
            text_understanding_similarity_list = []
            for rec_text_element in text_similarity_dict:
                rec_text_index = rec_text_element[0]
                marked_understanding = list(text_feat_competence_db.iloc[rec_text_index])[-1]
                text_similarity = rec_text_element[1]
                sum_similarity += text_similarity
                text_understanding_similarity_list.append([text_similarity, marked_understanding])
            if debug:print(text_understanding_similarity_list)
                
            #normalize similarity
            for similarity_ind in range(len(text_understanding_similarity_list)):
                text_understanding_similarity_list[similarity_ind][0] /=sum_similarity
            if debug:print(text_understanding_similarity_list)
                
            #math probability of sentence understanding
            math_prob = 0
            for similarity_ind in range(len(text_understanding_similarity_list)):
                m_e = float(text_understanding_similarity_list[similarity_ind][0])
                mark = float(text_understanding_similarity_list[similarity_ind][1])
                math_prob +=  m_e* mark
                if debug: print(text_understanding_similarity_list[similarity_ind][0], text_understanding_similarity_list[similarity_ind][1], m_e* mark)
            if debug: print("final",math_prob)
            return math_prob
        else:
            return 0
text_recommendation_from_marked_dataset("./texts_features_json/text_0.json", text_db, debug = False)

0.5499385780058443

In [82]:
answer_dict = {"222" :['0-' ,'1-' ,'2+' ,'4+'],
"862":['1+','2-', '4+', '6-', '12-'],
"321":['5+' ,'0-' ,'4-' ,'1+'],
"364":['2+', '6-', '3-', '7-' ],
"502":['13-', '3+', '0-', '4+'],
"878":['3-', '0+', '4-', '1-'],
"666":['3-', '1-', '2-' ],
"92":['1+', '2-'],
"615":['1+', '2-'],
"450":['6-', '7-', '11+', '9+' ,'10+' ],
"722":['1+', '6-', '2+', '3+', '6-', '8-'],
"732":['8-', '0-', '3-', '4-', '2+', '1-'],
"611":['1+'],
"251":['5-', '4+', '2+', '1+', '3+']}


In [89]:
%%time
a =2

Wall time: 1 ms


In [129]:
def calc_dev_from_eighty_percent(understanding_vector):
    diff_squared = 0
    for value in understanding_vector:
        diff_squared += (value - 0.8) ** 2
        #print(value - 0.8, (value - 0.8) ** 2,diff_squared )
    diff_squared /= 3
    #print(diff_squared)
    st_dev = math.sqrt(diff_squared)
    return st_dev
calc_dev_from_eighty_percent(text_recommendation_vector_dict[0])

0.24949928655405482

In [130]:
%%time
text_recommendation_vector_dict = {}
text_recommendation_dict = {}
for text_ind in tqdm(range(len(db_3000))):
    if str(text_ind) not in list(answer_dict.keys()):
        trigramms_path = "./words_features_json/text_" + str(text_ind) + ".json"
        sent_path = "./sentence_features_json/text_" + str(text_ind) + ".json"
        text_path = "./texts_features_json/text_" + str(text_ind) + ".json"
        trigr_recommendation = trigramm_recommendation_from_marked_dataset(trigramms_path, trigramms_db)
        sentence_recommendation = sentence_recommendation_from_marked_dataset(sent_path, sentence_db)
        text_recommendation = text_recommendation_from_marked_dataset(text_path, text_db)
        text_standard_deviation = calc_dev_from_eighty_percent([trigr_recommendation, sentence_recommendation, text_recommendation])
        text_recommendation_vector_dict[text_ind] = [trigr_recommendation, sentence_recommendation, text_recommendation]
        text_recommendation_dict[text_ind] = text_standard_deviation
sorted_text_feat_dict = sorted(text_recommendation_dict.items(), key=operator.itemgetter(1), reverse = False)

100%|████████████████████████████████████████████████████████████████████████████████| 898/898 [02:51<00:00,  5.23it/s]


Wall time: 2min 51s


In [113]:
sorted_text_feat_dict = sorted(text_recommendation_dict.items(), key=operator.itemgetter(1), reverse = False)
sorted_text_feat_dict[:5]

[(693, 0.1590651833868331),
 (279, 0.18096641885266368),
 (748, 0.1813461748178669),
 (194, 0.187156821930423),
 (629, 0.18936899042194447)]

# ИЗВЛЕКАЕМ ТЕКСТЫ ДЛЯ ВОПРОС

In [4]:
from ud_class import Model

In [5]:
model = Model('./udpipe parsers/russian-syntagrus-ud-2.0-170801.udpipe')

In [6]:
def get_conllu_from_unite_line_text(text, model):
    sentences = model.tokenize(text)
    for s in sentences:
        model.tag(s)
        model.parse(s)
    conllu = model.write(sentences, "conllu")
    return conllu

In [7]:
def get_conllu_text_map(conllu_parsed_object):
    conllu_text_map = []
    conllu_sentence_map = []
    for line in conllu_parsed_object.split('\n'):
        if line:
            if line[0].isdigit():
                #print(line.split('\t'))
                conllu_sentence_map.append(line.split('\t'))
            else:
                if(len(conllu_sentence_map) > 0):
                    conllu_text_map.append(conllu_sentence_map)
                    conllu_sentence_map = []   
                    #print("appended")
    if(len(conllu_sentence_map) > 0):
        conllu_text_map.append(conllu_sentence_map)
    return conllu_text_map

In [8]:
def get_lemm_and_orig_text_from_udmap(conllu_map):
    lemm_sentences_list = []
    sentences_list = []
    for sentence in conllu_map:
        lemm_line = ''
        line = ''
        for word in sentence: 
            if (word[3] != 'PUNCT'):
                #print(word[2])
                lemm_line += word[2] + ' '
                line += word[1] + ' '
        
        lemm_sentences_list.append(lemm_line.strip())
        sentences_list.append(line.strip())
        #print()
    return lemm_sentences_list, sentences_list

In [9]:
def read_text(path):
    raw_text = ''
    with open (path, 'r', encoding = "utf-8") as f:
        for line in f.readlines():
            raw_text += line + ' ' 
    return raw_text

In [125]:
conllu = get_conllu_from_unite_line_text(db_3000.iloc[int(222)]['texts_3000'], model)
conllu_text_map = get_conllu_text_map(conllu)
lemm_sentences,sentences_list = get_lemm_and_orig_text_from_udmap(conllu_text_map)
sentences_list

['В пятницу представители Северной Кореи признали что их ВМФ также понес потери в результате столкновения с военными кораблями Южной Кореи сообщает AP',
 'Неожиданная перестрелка между кораблями Северной и Южной Корей произошедшая 29 июня в результате безрассудной вооруженной провокации со стороны судов Южной Кореи привела к человеческим потерям с обеих сторон и потере одного военного корабля говорится в заявлении Центрального новостного агентства Северной Кореи',
 'Это первое признание Северной Кореей своих потерь однако величину их она предпочла оставить в секрете',
 'Ранее представители Южной Кореи сообщили что в ходе столкновения потеряли четырех своих моряков',
 'По их подсчетам потери Северной Кореи должны были составить не менее 30 человек']

In [126]:
for text_ind in list(answer_dict.keys()):
    text = db_3000.iloc[int(text_ind)]['texts_3000']
    conllu = get_conllu_from_unite_line_text(text, model)
    conllu_text_map = get_conllu_text_map(conllu)
    lemm_sentences,sentences_list = get_lemm_and_orig_text_from_udmap(conllu_text_map)
    print(sentences_list)
    text_path = "./texts_for_questions/text_" + text_ind + ".txt"
    with open(text_path, "w", encoding = "utf-8") as f:
        for sentence_ind in range(len(sentences_list)):
            f.write(str(sentence_ind) + ' ' + sentences_list[sentence_ind] + '\n')

['В пятницу представители Северной Кореи признали что их ВМФ также понес потери в результате столкновения с военными кораблями Южной Кореи сообщает AP', 'Неожиданная перестрелка между кораблями Северной и Южной Корей произошедшая 29 июня в результате безрассудной вооруженной провокации со стороны судов Южной Кореи привела к человеческим потерям с обеих сторон и потере одного военного корабля говорится в заявлении Центрального новостного агентства Северной Кореи', 'Это первое признание Северной Кореей своих потерь однако величину их она предпочла оставить в секрете', 'Ранее представители Южной Кореи сообщили что в ходе столкновения потеряли четырех своих моряков', 'По их подсчетам потери Северной Кореи должны были составить не менее 30 человек']
['Марсоход НАСА Curiosity обнаружил новые доказательства существования озера на Красной планете', 'Об этом ТАСС сообщили представители американской Лаборатории реактивного движения ЛРД в Пасадене штат Калифорния', 'Собранные результаты исследова

# тексты для искусственной рекоммендации

In [12]:
for file in os.listdir("/Users/lilyakhoang/Autotutor/processing_test_recommend/test_one_texts"):
    path = os.path.join("/Users/lilyakhoang/Autotutor/processing_test_recommend/test_one_texts",file )
    text = read_text(path)
    conllu = get_conllu_from_unite_line_text(text, model)
    conllu_text_map = get_conllu_text_map(conllu)
    lemm_sentences,sentences_list = get_lemm_and_orig_text_from_udmap(conllu_text_map)
    #print(sentences_list)
    text_path = "./test_one_texts/" + file[:-4] + "_parsed.txt"
    with open(text_path, "w", encoding = "utf-8") as f:
        for sentence_ind in range(len(sentences_list)):
            f.write(str(sentence_ind) + ' ' + sentences_list[sentence_ind] + '\n')

['В классе был урок русского языка', 'Преподаватель читал новый текст', 'Студенты внимательно слушали', 'Извините можно войти', 'Я опоздал сказал Джон', 'Что случилось Джон', 'Вчера вы опоздали сегодня вы опоздали', 'Нельзя опаздывать на урок', 'Вечером Джон пришел домой', 'Дома он долго думал «Вчера я опоздал сегодня я тоже опоздал', 'Это очень плохо', 'Не буду больше опаздывать на урок', 'Завтра я встану рано в 8 часов быстро сделаю зарядку приготовлю завтрак позавтракаю повторю домашнее задание', 'Завтра я не опоздаю Утром Джон встал в 8 часов', 'Он сделал зарядку приготовил завтрак позавтракал повторил домашнее задание и пошел в институт', 'В 9 часов 30 минут Джон пришел в институт', 'Что случилось молодой человек', 'Куда вы идете спросил дежурный', 'Я иду на урок', 'Сегодня я не опоздал ответил Джон', 'Идите домой молодой человек', 'Сегодня нет урока', 'Сегодня воскресенье']
['lix 0.2933 ttr 0.53 sent_properties 0.0 0 .0 0.02072917426099043 0 .0 0 .0 0.03214494674560842 0 .0 sente

['Таймырский краеведческий музей Дудинка Красноярский край представил уникальную коллекцию артефактов шаманской культуры народов севера на выставке Таймыр', 'Гений места которая начала свою работу на площадке Всероссийского музея декоративного искусства в Москве', 'Об этом сообщила в среду ТАСС директор таймырского музея Ольга Корнеева', 'Выставка начала свою работу сегодня на ней представлены более 200 наших экспонатов', 'Среди них есть очень редкие связанные с культами народов севера это шаманские предметы культа идолы погребальные маски элементы одежды последнего шамана Таймыра Тубяка Костеркина на Таймыре нет шаманов более четверти века прим.', 'ТАСС рассказала Корнеева']


KeyboardInterrupt: 

In [ ]:
for text_ind in rage(answer_dict.keys()):
    text = db_3000.iloc[int(text_ind)]['texts_3000']
    conllu = get_conllu_from_unite_line_text(text, model)
    conllu_text_map = get_conllu_text_map(conllu)
    lemm_sentences,sentences_list = get_lemm_and_orig_text_from_udmap(conllu_text_map)
    print(sentences_list)
    text_path = "./texts_for_questions/text_" + text_ind + ".txt"
    with open(text_path, "w", encoding = "utf-8") as f:
        for sentence_ind in range(len(sentences_list)):
            f.write(str(sentence_ind) + ' ' + sentences_list[sentence_ind] + '\n')

In [ ]:
def text_features_precalculation_initial (rec_text_map, marked_text_db):
    rec_text_text_similarity_dict = {}
    rec_text_features = []
    rec_text_features.append(rec_text_map['lix'])
    rec_text_features.append(rec_text_map['ttr'])
    rec_text_features.extend(rec_text_map['sent_properties'])
    rec_text_vec = np.array(rec_text_features).reshape(1, -1)
    #print(rec_text_vec)
    text_similarity_dict = {}
    for text_similarity_ind in range(len(marked_text_db)):
        text_vec = marked_text_db.iloc[text_similarity_ind][:-1].values.reshape(1, -1)
        #print(text_vec)
        similarity = cosine_similarity(rec_text_vec,text_vec)
        #print(similarity)
        text_similarity_dict[text_similarity_ind] = similarity[0][0]
    sorted_text_feat_dict = sorted(text_similarity_dict.items(), key=operator.itemgetter(1), reverse = True)
    text_feeat_mean_dict = []
    return sorted_text_feat_dict[:min(5, len(sorted_text_feat_dict))]
text_features_precalculation(potential_recommended_text_map, text_db)        

In [ ]:
def sent_features_recommendation_initial (rec_text_map, marked_sentences_db):
    sentences_understanding = {}
    sentence_map = rec_text_map['sentences_map']
    for sentence_ind in range(len(sentence_map)):
        rec_sent_feat = []
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['negation'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['coreference'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['vozvr_verb'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['prich'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['deepr'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['case_complexity'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['mean_depend_length'])
        rec_sent_vec = np.array(rec_sent_feat).reshape(1, -1)
        sent_similarity_dict = {}
        for sent_ind in range(len(marked_sentences_db)):
            sent_vec = marked_sentences_db.iloc[sent_ind][:-1].values.reshape(1, -1)
            similarity = cosine_similarity(rec_sent_vec,sent_vec)
            #print(similarity)
            sent_similarity_dict[sent_ind] = similarity[0][0]
        sorted_sent_similarity_dict = sorted(sent_similarity_dict.items(), key=operator.itemgetter(1), reverse = True)
        filtered = filter_similar(sorted_sent_similarity_dict)
        
        """
        sent_similarity_list = []
        for sim_sent in sorted_sent_similarity_dict[:3]:
            print(sim_sent)
            sent_undestanding_mark = list(marked_sentences_db.iloc[sim_sent[0]])[-1]
            sent_similarity_list.append(sent_undestanding_mark)
        """
        sentences_understanding[sentence_ind] = filtered
    return sentences_understanding
sent_features_recommendation(potential_recommended_text_map, sent_dp)        

In [ ]:
def trigramms_recommendation_intial (rec_text_map, marked_trigramms_db, trigrmms_list):
    sentences_map = rec_text_map['sentences_map']
    calc_text_dict = {}
    for sentence_ind in tqdm(range(len(sentences_map))):
        for word in sentences_map[sentence_ind]['sentence_words']:
            text_word_vec = word['lex_vector'].reshape(1, -1)
            text_word_dist_dict = {}
            for word_db_ind in range(len(marked_trigramms_db)):
                marked_trigramms_db_word_vec = marked_trigramms_db.iloc[word_db_ind][:-1].values.reshape(1, -1)
                similarity = cosine_similarity(text_word_vec,marked_trigramms_db_word_vec)
                text_word_dist_dict[word_db_ind] = similarity
           
            sorted_x = sorted(text_word_dist_dict.items(), key=operator.itemgetter(1), reverse = True)
            filtered_sorted_list = filter_similar_trigramms(sorted_x)
            if not filtered_sorted_list:
                calc_text_dict[word['lex_trigram']] = None
            else:
                similar_trigrams= []
                for similar_el in filtered_sorted_list:
                    similar_trigrams.append((similar_el, trigrmms_list[similar_el[0]],list(marked_trigramms_db.iloc[similar_el[0]])[-1] ))
                calc_text_dict[word['lex_trigram']] = similar_trigrams
                #=====ЗДЕСЬ ПР ПОДГРУЗКЕ НАДО БУДЕТ РАЗМЕТИТЬ ЦЕЛЕВУЮ ПЕРЕМЕННУЮВ СООТВТЕТСВИИ С КОРРЕКТНОСТЬЮ ОТВЕТА И ДАЛЕЕ ПЛЯСАТЬ
                
    vocab_recommendation_dict = {}
    for key in calc_text_dict.keys():
        print(key , "===>", calc_text_dict[key],'\n')
        understdanding_prob_list = []
        if (calc_text_dict[key]):
            for trgrm in calc_text_dict[key]:
                understdanding_prob = trgrm[2]
                understdanding_prob_list.append(understdanding_prob)
            understdanding_prob_mean = statistics.mean(understdanding_prob_list) 
            vocab_recommendation_dict[key] = understdanding_prob
    positive_understanding_prob = 0
    whole_understading_prob = 0 
    for key in vocab_recommendation_dict.keys():
        print(key , "===>", vocab_recommendation_dict[key],'\n')
        if (vocab_recommendation_dict[key] > 0):
            positive_understanding_prob += vocab_recommendation_dict[key]
            whole_understading_prob += abs(vocab_recommendation_dict[key])
        elif(vocab_recommendation_dict[key] < 0):
            whole_understading_prob += abs(vocab_recommendation_dict[key])
    understanding_prob = positive_understanding_prob/whole_understading_prob
    print("understanding_prob",understanding_prob)
        
trigramms_recommendation(potential_recommended_text_map,word_dp,  trigramms)